In [1]:
import numpy as np

def wer_score(hyp, ref, print_matrix=False):
  N = len(hyp)
  M = len(ref)
  L = np.zeros((N,M))
  for i in range(0, N):
    for j in range(0, M):
      if min(i,j) == 0:
        L[i,j] = max(i,j)
      else:
        deletion = L[i-1,j] + 1
        insertion = L[i,j-1] + 1
        sub = 1 if hyp[i] != ref[j] else 0
        substitution = L[i-1,j-1] + sub
        L[i,j] = min(deletion, min(insertion, substitution))
        # print("{} - {}: del {} ins {} sub {} s {}".format(hyp[i], ref[j], deletion, insertion, substitution, sub))
  if print_matrix:
    print("WER matrix ({}x{}): ".format(N, M))
    print(L)
  return int(L[N-1, M-1])

In [10]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 432 kB/s eta 0:00:01
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8917 sha256=a7918dad640c1ae3ec15e5346b587afcf4a1c27ae09301ca6dd1fa7e88a1eb69
  Stored in directory: /Users/karthikeyan/Library/Caches/pip/wheels/6e/df/38/abda47b884e3e25f9f9b6430e5ce44c47670758a50c0c51759
Successfully built et-xmlfile


In [2]:
wer_score("காலை வணக்கம் ஐயா ","வணக்கம் ஐயா",
          print_matrix = True)

WER matrix (17x11): 
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 1.  1.  2.  3.  4.  5.  6.  7.  8.  9.  9.]
 [ 2.  2.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 3.  3.  3.  3.  4.  5.  6.  7.  8.  9. 10.]
 [ 4.  4.  4.  4.  4.  5.  6.  6.  7.  8.  9.]
 [ 5.  5.  5.  5.  5.  5.  6.  7.  7.  8.  9.]
 [ 6.  5.  6.  6.  6.  6.  6.  7.  8.  8.  9.]
 [ 7.  6.  5.  6.  6.  7.  7.  7.  8.  9.  9.]
 [ 8.  7.  6.  5.  6.  7.  7.  8.  8.  9. 10.]
 [ 9.  8.  7.  6.  5.  6.  7.  8.  9.  9. 10.]
 [10.  9.  8.  7.  6.  5.  6.  7.  8.  9. 10.]
 [11. 10.  9.  8.  7.  6.  5.  6.  7.  8.  9.]
 [12. 11. 10.  9.  8.  7.  6.  5.  6.  7.  8.]
 [13. 12. 11. 10.  9.  8.  7.  6.  5.  6.  7.]
 [14. 13. 12. 11. 10.  9.  8.  7.  6.  5.  6.]
 [15. 14. 13. 12. 11. 10.  9.  8.  7.  6.  5.]
 [16. 15. 14. 13. 12. 11. 10.  9.  8.  7.  6.]]


6

In [3]:
!pip install rouge

In [4]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"


hypothesis = "காலை வணக்கம் ஐயா"
reference = "மாலை வணக்கம் ஐயா "
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
print(scores)

[{'rouge-1': {'f': 0.6666666616666668, 'p': 0.6666666666666666, 'r': 0.6666666666666666}, 'rouge-2': {'f': 0.4999999950000001, 'p': 0.5, 'r': 0.5}, 'rouge-l': {'f': 0.6666666616666668, 'p': 0.6666666666666666, 'r': 0.6666666666666666}}]


In [5]:
from nltk.translate.bleu_score import sentence_bleu
bleu_reference = [hypothesis]
candidate = reference
score = sentence_bleu(bleu_reference, candidate)
print(score)

0.8702397637697912


In [6]:
import nltk.translate.meteor_score as ms

ms.meteor_score(bleu_reference,candidate)

0.625

In [17]:
referenceFile = "./EnTa/corpus.bcn.dev.ta"
candidateFile = "./EnTa/microsoft_en_ta_results.ta"

VERBOSELEVEL = 0

import pandas as pd 
dict = {'ReferenceTxt':['sample text'], 
        'CandidateTxt':['sample translation'], 
        'Meteor':[0] ,
        'Bleu':[0] ,
        'Rouge':[0] ,
        'WER':[0] ,
        'SRC_WORD_COUNT':[0],
        'TRANSRSLT_WORD_COUNT':[0]
        
       }
resultDf = pd.DataFrame(dict)

with open(referenceFile) as refFile,open(candidateFile) as canFile: 
    s_reference = refFile.readline()
    s_candidate = canFile.readline()
    count = 1
    while s_candidate :
    #while count < 5 :
        
        s_meteor = ms.meteor_score([s_reference],s_candidate)
        s_bleu = sentence_bleu([s_reference], s_candidate)
        s_rouge = rouge.get_scores(s_reference, s_candidate)
        s_wer = wer_score(s_reference,s_candidate)
        count = count + 1

        resultDf.loc[len(resultDf.index)] = [s_reference,s_candidate,s_meteor,s_bleu,s_rouge,s_wer,len(s_reference.split()),len(s_candidate.split())]
        
        s_reference = refFile.readline()
        s_candidate = canFile.readline()
    
        if VERBOSELEVEL>2 :
            print(s_reference + "<---->"+s_candidate)
            print(";;meteor-->"+str(s_meteor) + ";;bleu-->"+str(s_bleu)+";;rouge-->"+str(s_rouge)+";;WER-->"+str(s_wer)+"\n")

          
resultDf.to_excel("output_ta.xlsx")

In [16]:
len(hypothesis.split())

3